In [ ]:
#for fixing a bug on kaggle
!pip install --upgrade tensorflow-io 

In [ ]:
import tensorflow as tf

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt

from pathlib import Path
from collections import Counter

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping

from PIL import Image, ImageChops, ImageEnhance

strategy = tf.distribute.MirroredStrategy()

In [ ]:
img_dim = 512
batch_size = 16

In [ ]:
dataset_path_train = "/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/train"
dataset_path_val = "/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/valid"
dataset_path_test = "/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/test"
temp_path = "/kaggle/working/temp"
os.makedirs(temp_path, exist_ok=True)

In [ ]:
def apply_ela(image_path, output_path, temp_path=temp_path):
    temp_image_path = os.path.join(temp_path, "temp.jpg")
    image = Image.open(image_path)
    image.save(temp_image_path, "JPEG", quality=90)
    temp_image = Image.open(temp_image_path)
    ela_image = ImageChops.difference(image, temp_image)
    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    ela_image.save(output_path, "JPEG")
    os.remove(temp_image_path)


In [ ]:
def generate_ela_dataset(dataset_path, output_path):
    os.makedirs(output_path, exist_ok=True)
    fake_images_folder = os.path.join(dataset_path, "fake")
    real_images_folder = os.path.join(dataset_path, "real")
    output_fake_face_path = os.path.join(output_path, "fake")
    output_real_face_path = os.path.join(output_path, "real")
    os.makedirs(output_fake_face_path, exist_ok=True)
    os.makedirs(output_real_face_path, exist_ok=True)
    
    count_fake = 0
    count_real = 0
    for filename in os.listdir(fake_images_folder):
        if filename.endswith(".jpg"):
            count_fake+=1
            image_path = os.path.join(fake_images_folder, filename)
            output_image_path = os.path.join(output_fake_face_path, f"ela_{filename}")
            apply_ela(image_path, output_image_path)
            print(f'Processed {count_fake} fake image(s) from {dataset_path}   ' , end='', flush=True) 
            print("\r", end='', flush=True) 

    for filename in os.listdir(real_images_folder):
        if filename.endswith(".jpg"):
            count_real+=1
            image_path = os.path.join(real_images_folder, filename)
            output_image_path = os.path.join(output_real_face_path, f"ela_{filename}")
            apply_ela(image_path, output_image_path)
            print(f'Processed {count_real} real image(s) from {dataset_path}   ' , end='', flush=True) 
            print("\r", end='', flush=True) 


In [ ]:
def image_dataset_preprocessing_from_dir(dataset_path, batch_size=batch_size, img_dim=img_dim, dataset_name="train"):
    output_path = os.path.join("/kaggle/working", dataset_name)
    generate_ela_dataset(dataset_path, output_path)
    data = tf.keras.utils.image_dataset_from_directory(
        directory=output_path,
        labels="inferred",
        label_mode="binary",
        class_names=None,
        color_mode="rgb",
        batch_size=batch_size,
        image_size=(img_dim, img_dim),
        shuffle=True,
        seed=16,
        interpolation="bilinear",
        follow_links=False,
        crop_to_aspect_ratio=False
    )
    data = data.map(lambda x, y: (x/255, y))
    return data

In [ ]:
def plot_dataset_images(data):
    # 1 = REAL
    # 0 = FAKE
    data_iterator = data.as_numpy_iterator()
    batch = data_iterator.next()
    fig, ax = plt.subplots(ncols=4,figsize=(20,20))
    for idx, img in enumerate(batch[0][:4]):
        ax[idx].imshow(img)
        ax[idx].title.set_text("REAL" if batch[1][idx] == 1 else "FAKE")

In [ ]:
with strategy.scope():
    data_train = image_dataset_preprocessing_from_dir(dataset_path=dataset_path_train, dataset_name="train")
    data_val = image_dataset_preprocessing_from_dir(dataset_path=dataset_path_val, dataset_name="val")
    data_test = image_dataset_preprocessing_from_dir(dataset_path=dataset_path_test, dataset_name="test")

In [ ]:
class ConvBlock(layers.Layer):
    def __init__(self, filters, kernel_size, stride=1, padding='valid'):
        super().__init__()
        self.conv = Conv2D(filters, kernel_size, stride, padding)
        self.norm = BatchNormalization()
        self.relu = layers.LeakyReLU(0.2)
    
    def call(self, x):
        x = self.conv(x)
        x = self.norm(x)
        x = self.relu(x)
        return x

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
    ]
)

In [ ]:
# kernel_dim = 3
# strides = 1
# with strategy.scope():
#     model = keras.Sequential([        
#         layers.Input([img_dim, img_dim, 3]),
#         data_augmentation,
#         ConvBlock(16, 3),
#         MaxPooling2D(),

#         ConvBlock(32, 3),
#         MaxPooling2D(),

#         ConvBlock(64, 3),
#         MaxPooling2D(),
        
#         ConvBlock(128, 3),
#         MaxPooling2D(),

#         Flatten(),

#         Dense(256, activation='relu'),

#         Dense(1, activation='sigmoid')
#     ])
#     model.compile(optimizer='adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
with strategy.scope():
    model = keras.Sequential([
        layers.Input([img_dim, img_dim, 3]),
        data_augmentation,
        ConvBlock(32, 3),
        MaxPooling2D(),
        ConvBlock(64, 3),
        MaxPooling2D(),
        ConvBlock(128, 3),
        MaxPooling2D(),
        ConvBlock(256, 3),
        layers.AveragePooling2D(),
        Dropout(0.1),
        ConvBlock(512, 3),
        ConvBlock(512, 3),
        layers.AveragePooling2D(),
        Flatten(),
        Dropout(0.1),
        Dense(256, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
with strategy.scope():
    callbacks = [
        EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True, start_from_epoch=10)
    ]
    history = model.fit(
        data_train,
        validation_data=data_val,
        epochs=100,
        callbacks=callbacks
    )

In [ ]:
fig_acc=plt.figure()
plt.plot(history.history['accuracy'], color='teal', label='accuracy')
plt.plot(history.history['val_accuracy'], color='orange', label='val_accuracy')
fig_acc.suptitle('Accuracy', fontsize=20)
plt.ylim(ymin=0)  
plt.legend(loc="upper left")
plt.show()

In [ ]:
# fig_loss=plt.figure()
# plt.plot(history.history['loss'], color='teal', label='loss')
# plt.plot(history.history['val_loss'], color='orange', label='val_loss')
# fig_loss.suptitle('Loss', fontsize=20)
# plt.ylim(ymin=0)  
# plt.legend(loc="upper left")
# plt.show()

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
precision = Precision()
recall = Recall()
bin_accuracy = BinaryAccuracy()
for batch in data_test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    precision.update_state(y, yhat)
    recall.update_state(y, yhat)
    bin_accuracy.update_state(y, yhat)
print(f'Precision:{precision.result().numpy()}')
print(f'Recall:{recall.result().numpy()}')
print(f'Accuracy: {bin_accuracy.result().numpy()}')

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
saved_model = model.save('/kaggle/working/models/model_3_UKNOWN.h5')

In [ ]:

# from IPython.display import FileLink 
# FileLink(r'/kaggle/working/models/model_lightweight_73.h5')

In [ ]:
# tf.saved_model.save()

In [ ]:
# import shutil
# shutil.make_archive("Model_and_ELA", 'zip', "/kaggle/working/")

In [ ]:
# ! tensorflowjs_converter --input_format keras /kaggle/working/models/model6_65.h5 /kaggle/working/models/jsver